# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Download the dataset
# This cell has to run only once. 
# NO need to run every time you arrive on this notebook. 

import requests
import tarfile
import os
import shutil

# Define the URL and folder paths
url = "https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz"
folder_name = "flowers"
file_name = "flower_data.tar.gz"
file_path = os.path.join(folder_name, file_name)

# Remove the folder or symbolic link if it already exists (equivalent to `rm -rf flowers`)
try:
    if os.path.islink(folder_name) or os.path.isfile(folder_name):
        os.remove(folder_name)  # Remove the symbolic link or file
    elif os.path.isdir(folder_name):
        shutil.rmtree(folder_name)  # Remove the directory
    print(f"Removed existing {folder_name} folder/file/soft link, if any.")
except FileNotFoundError:
    pass  # If the file or directory does not exist, do nothing

# Create the folder
os.makedirs(folder_name)
print(f"Created folder: {folder_name}")

# Download the file
response = requests.get(url, stream=True)

# Save the file in the 'flowers' folder
with open(file_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

print(f"Downloaded {file_name} to {folder_name}")

# Extract the file in the 'flowers' folder
if file_path.endswith("tar.gz"):
    with tarfile.open(file_path, "r:gz") as tar:
        tar.extractall(path=folder_name)
        print(f"Extracted {file_name} to {folder_name}")

# Clean up by removing the tar.gz file after extraction
os.remove(file_path)
print(f"Removed the downloaded tar.gz file: {file_path}")

In [14]:
# Imports here
import torch                        # Using tensor and its operation for deep learning
import torchvision                  # Image processing and pre-training the models
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import json
import shutil
import torch.optim as optim

# some contrains
input_size = 25088
path_to_checkpoint = os.path.join('.','checkpoint.pth')
learning_rate = 0.001

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/master/torchvision/transforms.html#)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. This converts the values of each color channel to be between -1 and 1 instead of 0 and 1.

In [2]:
data_dir = os.path.join('.', 'flowers')
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

In [4]:
## I just choose my favorite numbers for all train, test, valid data.
# The whole point of this project is learn how to implement a model for Multi-Class Classification.
# So, I will remove all other train, test, valid category and only keep 10 classes
my_fav_class = [1, 2, 10, 15, 20, 22, 63, 64, 90, 92] 

def clean_directories(main_directory, my_fav_class):
    """
    Deletes subdirectories in the main directory that are not listed in my_fav_class.
    Args:
        main_directory (str): The path to the main directory containing subdirectories.
        my_fav_class (List[int]): A list of favorite class names (as integers).
    Returns:
        None
    """
    # Loop through subdirectories inside the main directory
    for subdir in os.listdir(main_directory):
        # print(subdir)
        subdir_path = os.path.join(main_directory, subdir)
        if os.path.isdir(subdir_path):
            # Check if the subdirectory name is numeric
            if subdir.isdigit():
                # Convert subdirectory name to integer and check if it's in my_fav_class
                if int(subdir) not in my_fav_class:
                    # Delete the subdirectory
                    shutil.rmtree(subdir_path)
                    print(f"Deleted: {subdir_path}")
            elif subdir in ['train', 'valid', 'test']:
                clean_directories(subdir_path, my_fav_class)
clean_directories(data_dir, my_fav_class)

In [5]:
## Apply transformation to train data: scaling, cropping, and flipping
# Apply transformations in pipeline: Rando scaling, cropping, and flipping
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),      # Randomly crop and resize to 224x224
    transforms.RandomHorizontalFlip(),      # Randomly flip the image horizontally
    transforms.ToTensor(),                  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])
# No need to scaling or rotate for validation, and testing data but we want the data have the same size 224x224
valid_test_transforms = transforms.Compose([
    transforms.Resize(256),                 # Resize the image to 256x256, a common used value
    transforms.CenterCrop(224),             # Center crop to 224x224, this is mandatory because our train data using this size
    transforms.ToTensor(),                  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])
# Load the datasets with ImageFolder
# Result is dictionary with key are train, valid, or test. 
# Value are 3 ImageFolder objects from torchvision.datasets
image_datasets = {
    'train': datasets.ImageFolder(root=train_dir, transform=train_transforms),
    'valid': datasets.ImageFolder(root=valid_dir, transform=valid_test_transforms),
    'test': datasets.ImageFolder(root=test_dir, transform=valid_test_transforms),
}

batch_size = 64 # Choose same value as 5.10 lesson

# Using the image datasets and the trainforms, define the dataloaders
# Result is dictionary with key are train, valid, or test.
# Value are 3 DataLoader objects from torch.utils.data
dataloaders = {
    'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True), # Train data should be shuffle
    'valid' : torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size, shuffle=False),
    'test' : torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [6]:
import json
import random

cat_to_name_file = os.path.join('.', 'cat_to_name.json')

with open(cat_to_name_file, 'r') as f:
    cat_to_name = json.load(f)
print(len(cat_to_name)) # It should be 102 at the original

if(len(cat_to_name) != len(my_fav_class)):
    # I only want to keep my_fav_class class only
    cat_to_name = {key: value for key, value in cat_to_name.items() if int(key) in my_fav_class}
    print(len(cat_to_name))  # Should be 10 now
    with open(cat_to_name_file, 'w') as json_file:
        json.dump(cat_to_name, json_file, indent=4)

10


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [7]:
## 1. Load a pre-trained network (vgg16)

from torchvision import models
from torchvision.models import VGG16_Weights

# Load the pre-trained model
model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
# Make sure parameters are frozen (during training, the weights of the pretrained layers will not be updated.)
for param in model.parameters():
    param.requires_grad = False

list(model.classifier.children())

[Linear(in_features=25088, out_features=4096, bias=True),
 ReLU(inplace=True),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=4096, bias=True),
 ReLU(inplace=True),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=1000, bias=True)]

In [8]:
## 2. Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
import torch.nn as nn

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Move the model to the GPU
model.to(device)

output_size = len(cat_to_name)

# Create the feedforward classifier using nn.Sequential
classifier = nn.Sequential(
    nn.Linear(input_size, 4096),                 # First fully connected layer
    nn.ReLU(),                              # Activation function
    nn.Dropout(p=0.5),                      # Dropout layer for regularization
    nn.Linear(4096, 4096),                  # Second fully connected layer
    nn.ReLU(),                              # Activation function
    nn.Dropout(p=0.5),                      # Dropout layer for regularization
    nn.Linear(4096, output_size)            # Output layer
)

# Replace the classifier
model.classifier = classifier
list(model.classifier.children())

CUDA is available. Using GPU.


[Linear(in_features=25088, out_features=4096, bias=True),
 ReLU(),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=4096, bias=True),
 ReLU(),
 Dropout(p=0.5, inplace=False),
 Linear(in_features=4096, out_features=10, bias=True)]

In [ ]:
## 3. Train the classifier layers using backpropagation using the pre-trained network to get the features
## 4. Track the loss and accuracy on the validation set to determine the best hyperparameters
import torch
import torch.optim as optim
import time

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

# Train
validation_loss = 0.0
model = model.to(device)
begin = time.time()
num_epochs = 5 # I run with epoch = 12 first but look like 5 is the best option
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in dataloaders['train']:
        imgs, labels = imgs.to(device), labels.to(device)       # Move data to GPU
        optimizer.zero_grad()                                   # Clear the gradients 
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}")

    # Validation loop (optional)
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for imgs, labels in dataloaders['valid']:
            imgs, labels = imgs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    label_width = max(len('Training Loss:'), len('Validation Loss:')) + 1
    print(f"{'Training Loss:':<{label_width}} {(running_loss/len(dataloaders['train'])):.4f}")
    validation_loss = valid_loss/len(dataloaders['valid'])
    print(f"{'Validation Loss:':<{label_width}} {validation_loss:.4f}")
    print('-' * 50)
time_elapsed = time.time() - begin
print(f'Training completed after {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
## Do validation on the test set
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for imgs, labels in dataloaders['test']:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)  # Get model predictions
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class (this is Multi-class classification so could use F.log_softmax or torch.max)
        total += labels.size(0)  # Count total samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

accuracy = correct / total  # Calculate accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Save the checkpoint
model.class_to_idx = image_datasets['train'].class_to_idx

# Create the checkpoint dictionary
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'validation_loss': validation_loss,
    'class_to_idx' : model.class_to_idx,
    'test_accuracy' : accuracy
}

# print(checkpoint)

# Save the checkpoint
torch.save(checkpoint, path_to_checkpoint)

print(f"Checkpoint {path_to_checkpoint} saved successfully!")

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [15]:
# Write a function that loads a checkpoint and rebuilds the model
# Load the checkpoint
checkpoint = torch.load(path_to_checkpoint)

# Load the pre-trained model
model_loaded = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
# Make sure parameters are frozen (during training, the weights of the pretrained layers will not be updated.)
for param in model.parameters():
    param.requires_grad = False

# Recreate the model architecture using nn.Sequential
model_loaded.classifier = nn.Sequential(
    nn.Linear(input_size, 4096),            # First fully connected layer
    nn.ReLU(),                              # Activation function
    nn.Dropout(p=0.5),                      # Dropout layer for regularization
    nn.Linear(4096, 4096),                  # Second fully connected layer
    nn.ReLU(),                              # Activation function
    nn.Dropout(p=0.5),                      # Dropout layer for regularization
    nn.Linear(4096, output_size)            # Output layer
)

# Load everything from the checkpint to loaded model
model_loaded.load_state_dict(checkpoint['model_state_dict'])
optimizer_loaded = optim.Adam(model.classifier.parameters(), lr=learning_rate)
optimizer_loaded.load_state_dict(checkpoint['optimizer_state_dict'])
num_epochs = checkpoint['epoch']
validation_loss = checkpoint['validation_loss']
class_to_idx = checkpoint['class_to_idx']
accuracy = checkpoint['test_accuracy']

print(f"Checkpoint loaded {path_to_checkpoint}")
print(f"Epoch {num_epochs}, Validation Loss {validation_loss:.2f}, Test Accuracy: {accuracy * 100:.2f}%")

Checkpoint loaded .\checkpoint.pth
Epoch 5, Validation Loss 0.21, Test Accuracy: 94.83%


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# My original image is 920x920
image_path = os.path.join('.', 'flowers/predict/92/scarlet-beebalm.jpg') 

def process_image(image_path:str):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # Process a PIL image for use in a PyTorch model
    preprocess = transforms.Compose([
        transforms.Resize(256),             # Resize the image to 256x256
        transforms.CenterCrop(224),         # Crop the center of the image to 224x224
        transforms.ToTensor(),              # Convert the image to a tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
    ])
    # Load the image
    image = Image.open(image_path)

     # Apply the transformations
    image = preprocess(image)
    
    # Convert to NumPy array and reorder dimensions
    image = image.numpy()  # Convert to NumPy array
    image = np.transpose(image, (1, 2, 0))  # Reorder dimensions: (C, H, W) to (H, W, C)
    
    return image

processed_img = process_image(image_path)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes